In [175]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5

In [1]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM as OCS
from tqdm import tqdm
from pyod.models.abod import ABOD
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, silhouette_samples
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [2]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 데이터 로딩 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로.
            transform (callable, optional): 샘플에 적용될 Optional transform.
        """
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['img_path'].iloc[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image

# 이미지 전처리 및 임베딩
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = CustomDataset(csv_file='./train.csv', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

In [4]:
# 사전 학습된 모델 로드
model = models.resnet50(pretrained=True)
model.eval()  # 추론 모드로 설정

# 특성 추출을 위한 모델의 마지막 레이어 수정
model = torch.nn.Sequential(*(list(model.children())[:-1]))

model.to(device)

# 이미지를 임베딩 벡터로 변환
def get_embeddings(dataloader, model):
    embeddings = []
    with torch.no_grad():
        for images in tqdm(dataloader):
            images = images.to(device)
            emb = model(images)
            embeddings.append(emb.cpu().numpy().squeeze())
    return np.concatenate(embeddings, axis=0)

train_embeddings = get_embeddings(train_loader, model)

C:\Users\gyoo4\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\gyoo4\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:16<00:00,  2.38s/it]


In [5]:
# Isolation Forest 모델 학습
clf = IsolationForest(random_state=42)
clf.fit(train_embeddings)

IsolationForest(random_state=42)

In [6]:
# 테스트 데이터에 대해 이상 탐지 수행
test_data = CustomDataset(csv_file='./test.csv', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

test_embeddings = get_embeddings(test_loader, model)
test_pred = clf.predict(test_embeddings)

# Isolation Forest의 예측 결과(이상 = -1, 정상 = 1)를 이상 = 1, 정상 = 0으로 변환
test_pred = np.where(test_pred == -1, 1, 0)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.98s/it]


In [7]:
from bayes_opt import BayesianOptimization

In [110]:
def ocs_eval(nu):
    model = OCS(
        kernel = 'rbf',
        nu=float(nu),
        gamma='auto',
        )
    model.fit(train_embeddings)
    label = model.predict(train_embeddings)
    label = np.where(label == -1, 1, 0)
    try:
        score = silhouette_score(train_embeddings, label)
    except ValueError:
        score = -1.0
    return score

# 하이퍼파라미터 범위 설정
pbounds = {
    'nu': (0.0000001, 0.2)
}

optimizer = BayesianOptimization(f=ocs_eval, pbounds=pbounds, random_state = 42)
optimizer.maximize(init_points=3, n_iter=20)

|   iter    |  target   |    nu     |
-------------------------------------
| 1         | 0.09492   | 0.07491   |
| 2         | 0.07005   | 0.1901    |
| 3         | 0.07777   | 0.1464    |
| 4         | 0.09      | 0.003979  |
| 5         | 0.1002    | 0.07967   |
| 6         | 0.09205   | 0.07965   |
| 7         | 0.08931   | 0.07971   |
| 8         | 0.09499   | 0.07496   |
| 9         | 0.09319   | 0.07487   |
| 10        | 0.09174   | 0.075     |
| 11        | 0.1013    | 0.07482   |
| 12        | 0.09856   | 0.0748    |
| 13        | 0.1059    | 0.07474   |
| 14        | 0.1013    | 0.07472   |
| 15        | 0.1018    | 0.07476   |
| 16        | 0.09748   | 0.07463   |
| 17        | 0.097     | 0.07455   |
| 18        | 0.1047    | 0.07449   |
| 19        | 0.09356   | 0.07444   |
| 20        | 0.09721   | 0.07452   |
| 21        | 0.06831   | 0.1874    |
| 22        | 0.08364   | 0.137     |
| 23        | 0.1039    | 0.0503    |


In [111]:
max_para = optimizer.max['params']

In [112]:
ocs = OCS(kernel='rbf',
          nu=max_para['nu'], 
          gamma = 'auto'
         ).fit(train_embeddings)

test_pred = ocs.predict(test_embeddings)
test_pred = np.where(test_pred == -1, 1, 0)

In [138]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = test_pred

submit.to_csv('./result_submit.csv', index=False)

In [23]:
from sklearn.model_selection import train_test_split
import keras

In [24]:
X_train, X_test = train_test_split(train_embeddings, test_size=0.2, random_state=42)

In [65]:
input_dim = X_train.shape[1]

encoder = keras.models.Sequential([
    keras.layers.Dense(400, activation='relu', input_shape=[input_dim]),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(10, activation='relu')
])

decoder = keras.models.Sequential([
    keras.layers.Dense(50, activation='relu', input_shape=[10]),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(400, activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(input_dim, activation='relu'),
])

autoencoder = keras.models.Sequential([encoder, decoder])

autoencoder.compile(
    loss='mean_squared_error', optimizer=keras.optimizers.Adam(), metrics=['mse'])

In [133]:
nb_epoch = 100
batch_size = 320
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.0145 - accuracy: 0.6824 - val_loss: 0.0196 - val_accuracy: 0.6744
Epoch 2/100
1/1 [==============================] - 0s 269ms/step - loss: 0.0164 - accuracy: 0.6882 - val_loss: 0.0185 - val_accuracy: 0.6977
Epoch 3/100
1/1 [==============================] - 0s 217ms/step - loss: 0.0156 - accuracy: 0.7471 - val_loss: 0.0185 - val_accuracy: 0.6744
Epoch 4/100
1/1 [==============================] - 0s 112ms/step - loss: 0.0155 - accuracy: 0.7235 - val_loss: 0.0187 - val_accuracy: 0.6744
Epoch 5/100
1/1 [==============================] - 0s 102ms/step - loss: 0.0149 - accuracy: 0.7176 - val_loss: 0.0188 - val_accuracy: 0.6977
Epoch 6/100
1/1 [==============================] - 0s 105ms/step - loss: 0.0151 - accuracy: 0.7059 - val_loss: 0.0189 - val_accuracy: 0.6977
Epoch 7/100
1/1 [==============================] - 0s 104ms/step - loss: 0.0149 - accuracy: 0.7412 - val_loss: 0.0188 - val_accuracy: 0.6977
Epoch 8/100
1/1 

In [134]:
autoencoder = load_model('model.h5')

In [135]:
pred = autoencoder.predict(test_embeddings)

4/4 [==============================] - 0s 6ms/step


In [136]:
threshold = history['loss'][max(history['accuracy']) == history['accuracy']]

loss = [0]*(test_embeddings.shape[0])

for i in range(test_embeddings.shape[0]):
        loss[i] = sum((test_embeddings[i] - pred[i])**2) / test_embeddings.shape[1]

test_pred = np.array(loss) > threshold

test_pred = test_pred.astype(int)

In [137]:
test_pred

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])